# Homework 3

1. Создать Dataset для загрузки данных (sklearn.datasets.fetch_california_housing)
1. Обернуть его в Dataloader
1. Написать архитектуру сети, которая предсказывает стоимость недвижимости. Сеть должна включать BatchNorm слои и Dropout (или НЕ включать, но нужно обосновать)
1. Сравните сходимость Adam, RMSProp и SGD, сделайте вывод по качеству работы модели train-test разделение нужно сделать с помощью sklearn random_state=13, test_size = 0.25

#### Создать Dataset для загрузки данных (sklearn.datasets.fetch_california_housing)

In [105]:
import torch
import torch.nn.functional as F
import torch.nn as nn
import numpy as np
from PIL import Image
from torchvision import transforms #, datasets
from sklearn import datasets
from sklearn.model_selection import train_test_split
from tqdm import tqdm

In [106]:
train_dataset = sklearn.datasets.fetch_california_housing(data_home='./data')

In [107]:
train_dataset.keys()

dict_keys(['data', 'target', 'frame', 'target_names', 'feature_names', 'DESCR'])

In [108]:
train_dataset.DESCR

'.. _california_housing_dataset:\n\nCalifornia Housing dataset\n--------------------------\n\n**Data Set Characteristics:**\n\n    :Number of Instances: 20640\n\n    :Number of Attributes: 8 numeric, predictive attributes and the target\n\n    :Attribute Information:\n        - MedInc        median income in block group\n        - HouseAge      median house age in block group\n        - AveRooms      average number of rooms per household\n        - AveBedrms     average number of bedrooms per household\n        - Population    block group population\n        - AveOccup      average number of household members\n        - Latitude      block group latitude\n        - Longitude     block group longitude\n\n    :Missing Attribute Values: None\n\nThis dataset was obtained from the StatLib repository.\nhttps://www.dcc.fc.up.pt/~ltorgo/Regression/cal_housing.html\n\nThe target variable is the median house value for California districts,\nexpressed in hundreds of thousands of dollars ($100,000

In [109]:
data = train_dataset.data
target = train_dataset.target

In [110]:
data.shape

(20640, 8)

In [111]:
target.shape

(20640,)

In [112]:
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.25, random_state=13)

In [113]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((15480, 8), (5160, 8), (15480,), (5160,))

In [147]:
class CaliforniaHousing(torch.utils.data.Dataset):
    def __init__(self, init_dataset, init_target):
        self._base_dataset = init_dataset
        self._target = init_target
    
    def __len__(self):
        return self._base_dataset.shape[0]
    
    def __getitem__(self, idx):
        return torch.from_numpy(self._base_dataset[idx]).float(), torch.tensor(self._target[idx]).float()

#### Обернуть его в Dataloader

In [169]:
dataset = CaliforniaHousing(X_train, y_train)
train_loader = torch.utils.data.DataLoader(dataset, batch_size=8, shuffle=True, num_workers=8, drop_last=False)

In [198]:
target_dataset = CaliforniaHousing(X_test, y_test)
test_loader = torch.utils.data.DataLoader(target_dataset, batch_size=5160, shuffle=True, num_workers=8, drop_last=False)

#### Написать архитектуру сети, которая предсказывает стоимость недвижимости. Сеть должна включать BatchNorm слои и Dropout (или НЕ включать, но нужно обосновать)

In [170]:
class Perceptron(nn.Module):
    def __init__(self, input_dim, output_dim, activation="relu"):
        super(Perceptron, self).__init__()
        self.fc = nn.Linear(input_dim, output_dim)
        self.activation = activation
        
    def forward(self, x):
        x = self.fc(x)
        if self.activation=="relu":
            return F.leaky_relu(x)
        if self.activation=="sigmoid":
            return F.sigmoid(x)
        raise RuntimeError
        

class FeedForward(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super(FeedForward, self).__init__()
        self.bn1 = nn.BatchNorm1d(input_dim)
        self.fc1 = Perceptron(input_dim, hidden_dim * 2, "relu")
        self.bn2 = nn.BatchNorm1d(hidden_dim * 2)
        self.dp2 = nn.Dropout(0.5)
        self.fc2 = Perceptron(hidden_dim * 2, hidden_dim, "relu")
        self.bn3 = nn.BatchNorm1d(hidden_dim)
        self.dp3 = nn.Dropout(0.33)
        self.fc3 = Perceptron(hidden_dim, 1, "relu")
        
    def forward(self, x):
        x = self.bn1(x)
        x = self.fc1(x)
        x = self.bn2(x)
        x = self.dp2(x)
        x = self.fc2(x)
        x = self.bn3(x)
        x = self.dp3(x)
        x = self.fc3(x)
        return x

#### Сравните сходимость Adam, RMSProp и SGD, сделайте вывод по качеству работы модели train-test разделение нужно сделать с помощью sklearn random_state=13, test_size = 0.25

In [200]:
optimizers = {
    'Adam': torch.optim.Adam,
    'RMSProp': torch.optim.RMSprop,
    'SGD': torch.optim.SGD
}

In [202]:
for key, opt in optimizers.items():
    print('Processing : {}'.format(key))
    net = FeedForward(8, 64)
    if key == 'SGD':
        optimizer = opt(net.parameters(), lr=0.01)
    else:
        optimizer = opt(net.parameters())
    criterion = nn.MSELoss()
    net.train()
    
    for epoch in tqdm(range(10)):  
        running_loss = 0.0
        for i, data in enumerate(train_loader, 0):
            inputs, labels = data[0], data[1]

            # обнуляем градиент
            optimizer.zero_grad()

            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            # выводим статистику о процессе обучения
            running_loss += loss.item()
            if i and i % 648 == 0:    # печатаем каждые 300 mini-batches
                print('[%d, %5d] loss: %.3f' %
                      (epoch + 1, i + 1, running_loss / 300))
                running_loss = 0.0

    print('Training is finished!')
    
    net.eval()
    test_iter = iter(test_loader)
    for it in test_iter:
        inputs, targets = it
        pred_target = net(inputs)
        target_loss = criterion(pred_target, targets)
        print('Test loss: {}'.format(target_loss))

Processing : Adam


  0%|          | 0/10 [00:00<?, ?it/s]

[1,   649] loss: 5.865
[1,  1297] loss: 3.295


 10%|█         | 1/10 [00:05<00:45,  5.08s/it]

[2,   649] loss: 3.026
[2,  1297] loss: 3.099


...

[10,   649] loss: 2.946
[10,  1297] loss: 2.891


100%|██████████| 10/10 [00:48<00:00,  4.83s/it]

Training is finished!



Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f30af361c10>
Traceback (most recent call last):
  File "/home/nevl/anaconda3/envs/learn3/lib/python3.9/site-packages/torch/utils/data/dataloader.py", line 1478, in __del__
    self._shutdown_workers()
  File "/home/nevl/anaconda3/envs/learn3/lib/python3.9/site-packages/torch/utils/data/dataloader.py", line 1461, in _shutdown_workers
    if w.is_alive():
  File "/home/nevl/anaconda3/envs/learn3/lib/python3.9/multiprocessing/process.py", line 160, in is_alive
    assert self._parent_pid == os.getpid(), 'can only test a child process'
AssertionError: can only test a child process
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f30af361c10>
Traceback (most recent call last):
  File "/home/nevl/anaconda3/envs/learn3/lib/python3.9/site-packages/torch/utils/data/dataloader.py", line 1478, in __del__
    self._shutdown_workers()
  File "/home/nevl/anaconda3/envs/learn3/lib/python3.9/site-

Test loss: 1.3566969633102417


In [203]:
"""
    Adam: Test loss: 1.3557902574539185
    RMSProp: Test loss: 1.352184772491455
    SGD: Test loss: 1.3566969633102417
"""

'\n    Adam: Test loss: 1.3557902574539185\n    RMSProp: Test loss: 1.352184772491455\n    SGD: Test loss: 1.3566969633102417\n'

По приведённому выше тесту самым быстрым обучение было при оптимизаторе SGD. Лучшие результаты получены у RMSProp, на вротом месте Adam, SGD на третьем месте. После лекции ожидания были, что Adam вырвется вперёд. Хотя может не достаточное обучение и где-нибудь через 10 эпох Adam победил бы.